<a href="https://colab.research.google.com/github/hank199599/data_science_from_scratch_reading_log/blob/main/Chapter19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 深度學習(Deep learning)
原本指的是「深度」神經網路，現被用來泛指各種神經網路。

# 張量
在神經網路函示庫中，n維陣列被稱為**張量(tensor)**  
理想情況下，可以使用：

```python
# 張量Tensor要不是一個浮點數，就是一個張量列表
Tensor = Union[float,List[Tensor]]
```


In [153]:
#如同我們說：
Tensor = list

### 輔助函式：找出張量的形狀

In [154]:
from typing import List

def shape(tensor:Tensor) ->List[int]:
  sizes:List[int] = []
  while isinstance(tensor,list):
    sizes.append(len(tensor))
    tensor = tensor[0]
  return sizes

In [155]:
assert shape([1,2,3]) == [3]

In [156]:
assert shape([[1,2],[3,4],[5,6]]) == [3,2]

由於張量可能具有任意數量的維度，  
因此在使用張量時，通常需要採用**遞迴**的作法。

In [157]:
def is_1d(tensor:Tensor) ->bool:
  """
  如果tensor[0]是一個列表，他就是一個高維張量
  否則就是一個一維向量
  """
  return not isinstance(tensor[0],list)

In [158]:
assert is_1d([1,2,3])

In [159]:
assert not is_1d([[1,2],[3,4]])

### 輔助函式：tensor_sum 函數

In [160]:
def tensor_sum(tensor:Tensor)->float:
  """把張量的所有值加總起來"""
  if is_1d(tensor):
    return sum(tensor) #只是一個浮點數列表，就用Python的sum函式
  else:
    return sum(tensor_sum(tensor_i) for tensor_i in tensor)

In [161]:
assert tensor_sum([1,2,3]) == 6

In [162]:
assert tensor_sum([[1,2],[3,4]]) == 10

### 輔助函式：把某個函數套用到單一張量中的每個元素

In [163]:
from typing import Callable

def tensor_apply(f:Callable[[float],float],tensor:Tensor) ->Tensor:
  """把函式f套用到每個元素"""
  if is_1d(tensor):
    return [f(x) for x in tensor]
  else:
    return [tensor_apply(f,tensor_i) for tensor_i in tensor]

In [164]:
assert tensor_apply(lambda x:x+1,[1,2,3]) == [2,3,4]

In [165]:
assert tensor_apply(lambda x:2*x,[[1,2],[3,4]]) == [[2,4],[6,8]]

### 輔助函式：依據張量形狀，建立另一個形狀一樣的零張量

In [166]:
def zero_like(tensor:Tensor) ->Tensor:
  return tensor_apply(lambda _: 0.0,tensor)

In [167]:
assert zero_like([1,2,3]) == [0,0,0]

In [168]:
assert zero_like([[1,2],[3,4]]) == [[0,0],[0,0]]

### 輔助函式：將某個函式套用到兩個張量

In [169]:
def temsor_combine(f:Callable[[float,float],float],t1:Tensor,t2:Tensor) ->Tensor:
  """把函式f套用到t1與t2的相應元素"""
  if is_1d(t1):
    return [f(x,y) for x,y in zip(t1,t2)]
  else:
    return [temsor_combine(f,t1_i,t2_i) for t1_i,t2_i in zip(t1,t2)]

In [170]:
import operator
assert temsor_combine(operator.add,[1,2,3],[4,5,6]) == [5,7,9]
assert temsor_combine(operator.mul,[1,2,3],[4,5,6]) == [4,10,18]

# 層的抽象概念

建立一種機制，能用來時做出各式各樣的神經網路。
最基本的概念是「Layer」。  
他知道如何把輸入套入某種函數，亦能夠進行**反向傳播**的方式計算梯度。
  
在真正的子類別中，forward 與 backward 方法都會進行實作，  
一旦建構了神經網路，我們就能以梯度遞減的方式來進行訓練。



In [171]:
from typing import Iterable,Tuple

class Layer:
  """
  我們的神經網路是由許多層組成，其中每一層都知道
  如何以正向傳播的方式對輸入進行某些計算
  以及如何以反向傳播的方式計算梯度
  """
  def forward(self,input):
    """
    可以注意到，這裡缺了型別可以設定。
    我們並不打算限定各層的輸入是什麼型別。
    也不限定return的輸入是甚麼型別
    """
    raise NotImplementedError
  
  def backward(self,gradient):
    """
    同樣地，我們並不打算限定梯度應該是什麼型別，
    這完全由你自己決定，
    只要確定合理即可
    """
    raise NotImplementedError
  
  def params(self)->Iterable[Tensor]:
    """
    返回此層的參數，預設的實作方式不會回送任何東西。
    如果你的Layer沒有任何參數，
    並不需要實做這個方法
    """
    return ()
  def grads(self) ->Iterable[Tensor]:
    """
    送回梯度，順序與params相同
    """
    return ()

### 範例：不須更新參數的sigmoid層：

In [172]:
from typing import List
import math

Vector = List[float]

def sigmoid(t:float)->float:
  return 1/(1+math.exp(-t))

class Sigmoid(Layer):

  def forward(self,input:Vector)->Tensor:
    """
    把sigmoid函式套用到輸入張量的每個元素，
    然後儲存結果，供反向傳播使用
    """
    self.sigmoids = tensor_apply(sigmoid,input)
    return self.sigmoids
  
  def backward(self,gradient:Tensor)->Tensor:
    return tensor_combine(lambda sig,grad:sig*(1-sig)*grad,self.sigmoids,gradient)


## 線性層
代表神經元的 **dot(weight,inputs)** 點積的部分
  
在這裡實作三種方式生成初始的隨機張量：
1. 自[0,1]的隨機分布中選擇初始值
2. 自標準常態分布中選擇初始值
3. 使用「Xavier initialization」，其中每個權重為平均是0，變異量為2/(num_inputs+num_outputs)的標準常態分布中選擇初始值


### 前置作業

In [173]:
import random
import math

SQRT_TWO_PI = math.sqrt(2* math.pi)

def normal_pdf(x:float,mu:float=0,sigma:float=1) ->float:
  return (math.exp(-(x-mu)**2/2/sigma**2)/(SQRT_TWO_PI*sigma))

def normal_cdf(x:float,mu:float=0,sigma:float=1)->float:
  return (1+math.erf((x-mu)/math.sqrt(2)/sigma))/2

def inverse_normal_cdf(p:float,
            mu:float = 0,
            sigma:float=1,
            tolerance:float=0.00001) -> float:
  # 如果不是標準常態分佈，就先轉換成標準常態分佈
  if mu != 0 or sigma != 1:
    return mu + sigma*inverse_normal_cdf(p,tolerance = tolerance)
  
  low_z = -10.0 # normal_cdf(-10)是(趨近於) 0
  hi_z =  10.0 # normal_cdf(10)是(趨近於) 1
  while hi_z - low_z > tolerance:  
    mid_z = (low_z + hi_z) / 2   # 計算出中間值
    mid_p = normal_cdf(mid_z)     # 以及累積分佈函數在該處所應對的值
    if mid_p < p :
      low_z = mid_z        #中間的值太低，就往上繼續搜尋
    else:
      hi_z = mid_z        #中間的值太高，就往下繼續搜尋
  
  return mid_z

In [174]:
def random_uniform(*dims:int) ->Tensor:
  if len(dims) == 1:
    return [random.random() for _ in range(dims[0])]
  else:
    return [random_uniform(*dims[1:]) for _ in range(dims[0])]

def random_normal(*dims:int,mean:float=0.0,variance:float=1.0) ->Tensor:
  if len(dims) == 1:
    return [mean + variance * inverse_normal_cdf(random.random()) for _ in range(dims[0])]
  else:
    return [random_normal(*dims[1:],mean=mean,variance=variance) for _ in range(dims[0])]
  

In [175]:
assert shape(random_uniform(2,3,4)) == [2,3,4]

In [176]:
assert shape(random_normal(5,6,mean=10)) == [5,6]

將他們包裝在random_tensor函式中

In [177]:
def random_tensor(*dims:int,init:str = 'normal')->Tensor:
  if init == 'normal':
    return random_normal(*dims)
  elif init == 'unifrom':
    return random_uniform(*dims)
  elif init == 'xavier':
    variance = len(dims)/sum(dims)
    return random_normal(*dims,variance=variance)
  else:
    raise ValueError(f"unknown init: {init}")

### 定義線性層

In [178]:
def dot(v:Vector,w:Vector)->float:
  #計算v_1*w_1+... +v_n*w_n
  assert len(v)==len(w),"兩個向量必須有相同的維度"

  return sum(v_i*w_i for v_i,w_i in zip(v,w))

In [179]:
class Linear(Layer):
  def __init__(self,input_dim:int,output_dim:int,init:str='xavier')->None:
    """
    此層具有output_dims個神經元，每個神經元都有 imnput_dims個權重以及一個偏差量
    """
    self.input_dim = input_dim
    self.output_dim = output_dim

    # self.w[o] 是第 o 個神經元的權重
    self.w = random_tensor(output_dim,input_dim,init=init)

    # self.w[o] 是第 o 個神經元的偏差項
    self.b = random_tensor(output_dim,init=init)
  
  """
  每個神經元都有一個輸出，將它保存在向量中
  = 輸入和權重的點積 + 偏差值
  """
  def forward (self,input:Tensor) ->Tensor:
    #保存以供反向傳遞使用
    self.input = input

    #送回神經元的輸出向量
    return [dot(input,self.w[o])+self.b[o] for o in range(self.output_dim)]

  def backward(self,gradient:Tensor) ->Tensor:
    #　每個　b[o]　都會被加到　output[o]　之中
    #　亦即ｂ的梯度等於　output的梯度
    self.b_grad = gradient

    # 每個w[o][i]都會乘以input[i]再加到output[o]之中
    # 因此梯度是 input[i]*gradient[o]
    self.w_grad = [[self.input[i]*gradient[o] for i in range(self.input_dim)] for o in range(self.output_dim)]

    # 每個input[i]都會與每個w[o][i]相乘，再加到每個output[o]之中
    # 因此其梯度就是橫跨所有outputs
    # w[o][i]*gradient[o]加總之和
    return [sum(self.w[o][i]* gradient[o] for o in range(self.output_dim)) for i in range(self.input_dim)]
  
  def paramas(self) ->Iterable[Tensor]:
    return [self.w,self,b]
  
  def grads(self)->Iterable[Tensor]:
    return [self.w_grad,self,b_grad]

# 把神經網路視為一系列的層

In [180]:
from typing import List

def Sequential(Layer):
  """
  一行就包括一系列的其他層。
  每一層的輸出做為下一層的輸入，
  """
  def __init__(self,layers:List[Layer])->None:
    self.layers = layers
  
  def forward(self,input):
    """讓輸入照順序正向通過每一層"""
    for layer in self.layers:
      input = layer.forward(input)
    return input
  
  def backward(self,gradient):
    """讓梯度反向通過每一層以進行反向傳播"""
    for layer in reversed(self.layers):
      gradient = layer.backward(gradient)
    return gradient
  
  def params(self) ->Iterable[Tensor]:
    """送回每一層的參數"""
    return (param for layer in self.layers for param in layers.params())

  def grads(self) ->Iterable[Tensor]:
    """送回每一層的梯度"""
    return (grad for layer in self.layers for grad in layers.grads())

將XOR神經網路表示為：

In [181]:
xor_net=Sequential([
  Linear(input_dim=2,output_dim=2),
  Sigmoid(),
  Linear(input_dim=2,output_dim=1),
  Sigmoid()
])

TypeError: ignored

# 損失與最佳化

# 範例：XOR再次嘗試

# 其他激活函數

# **範例**：FizzBuzz 再次嘗試

# Softmax與交叉熵

# Dropout隨機拋棄

# 範例：MNIST

# 模型的儲存與載入